# OMC optical layout for O4

## Import modules

In [1]:
import gtrace
import gtrace.beam as beam
import gtrace.optcomp as opt
import gtrace.optics.gaussian as gauss
import gtrace.draw as draw
import gtrace.draw.renderer as renderer
import gtrace.draw.tools as tools
from gtrace.unit import *
from gtrace.optics.geometric import vector_normalize as normalize
from gtrace.optics.geometric import vector_rotation_2D
from gtrace.nonsequential import non_seq_trace

import scipy.optimize as sopt

import _pickle

## Units

In [2]:
ppm = 1e-6
nm = 1e-9
m = 1
percent = 0.01

MHz = 1e6

from scipy.constants import c

## Parameters

In [3]:
OBS3_curved = True

OMC1_HR_Refl = 0.996
OMC3_HR_Refl = 99.996*percent
nsilica = 1.44967
Roc = 1.8
AOI = deg2rad(3.0)

## Utility Functions

In [4]:
def align_OMC():
    '''
    Given OMC mirrors, adjust their orientations to form a cavity.
    '''
    v1 = OMC2.HRcenter - OMC1.HRcenter
    v1 = v1/np.linalg.norm(v1)
    v2 = OMC4.HRcenter - OMC1.HRcenter
    v2 = v2/np.linalg.norm(v2)
    OMC1.normVectHR = (v1+v2)/2

    v1 = OMC1.HRcenter - OMC2.HRcenter
    v1 = v1/np.linalg.norm(v1)
    v2 = OMC3.HRcenter - OMC2.HRcenter
    v2 = v2/np.linalg.norm(v2)
    OMC2.normVectHR = (v1+v2)/2

    v1 = OMC2.HRcenter - OMC3.HRcenter
    v1 = v1/np.linalg.norm(v1)
    v2 = OMC4.HRcenter - OMC3.HRcenter
    v2 = v2/np.linalg.norm(v2)
    OMC3.normVectHR = (v1+v2)/2

    v1 = OMC1.HRcenter - OMC4.HRcenter
    v1 = v1/np.linalg.norm(v1)
    v2 = OMC3.HRcenter - OMC4.HRcenter
    v2 = v2/np.linalg.norm(v2)
    OMC4.normVectHR = (v1+v2)/2

In [5]:
def OMC_eigen_mode():
    '''
    Compute the eigen mode of the OMC.
    Returns the q-parameters and a beam object of the obtained eigen mode at the HR surface of OMC1.
    '''
    q0 = gauss.Rw2q(np.inf, 410*um)
    b = beam.GaussianBeam(q0=q0, wl=1064*nm)

    v1 = OMC2.HRcenter - OMC1.HRcenter
    b.pos = OMC1.HRcenter
    b.dirVect = v1
    
    beams = OMC2.hitFromHR(b, order=1)
    b = beams['r1']
    
    beams = OMC3.hitFromHR(b, order=1)
    b = beams['r1']    

    beams = OMC4.hitFromHR(b, order=1)
    b = beams['r1']

    beams = OMC1.hitFromHR(b, order=1)
    b = beams['r1']    
    
    #Eigenmode
    M = b.Mx
    A = M[0,0]
    B = M[0,1]
    C = M[1,0]
    D = M[1,1]
    qOMCx = 1.0/((D-A)/(2*B)-1j*np.sqrt(4-(A+D)**2)/(2*B))

    M = b.My
    A = M[0,0]
    B = M[0,1]
    C = M[1,0]
    D = M[1,1]
    qOMCy = 1.0/((D-A)/(2*B)-1j*np.sqrt(4-(A+D)**2)/(2*B))

    # Propagate the eigenmode through the OMC
    v1 = OMC2.HRcenter - OMC1.HRcenter
    b = beam.GaussianBeam(q0x=qOMCx, q0y=qOMCy, wl=1064*nm, pos=OMC1.HRcenter, dirVect=v1)

    beams = OMC2.hitFromHR(b, order=1)
    b = beams['r1']
    
    beams = OMC3.hitFromHR(b, order=1)
    b = beams['r1']    

    beams = OMC4.hitFromHR(b, order=1)
    b = beams['r1']

    beams = OMC1.hitFromHR(b, order=1)
    b = beams['r1']    
    
    return (qOMCx, qOMCy, b)

In [6]:
def OMC_beams():
    '''
    Propagate the eigen mode beam in the OMC and returns a dictionary containing beam objects in the OMC.
    '''

    beam_dict = {}

    (qOMCx, qOMCy, b) = OMC_eigen_mode()

    beams = OMC2.hitFromHR(b, order=1)
    b = beams['r1']

    beam_dict['OMC1_to_OMC2'] = beams['input']
    
    beams = OMC3.hitFromHR(b, order=1)
    b = beams['r1']   

    beam_dict['OMC2_to_OMC3'] = beams['input']

    beams = OMC4.hitFromHR(b, order=1)
    b = beams['r1']
    beam_dict['OMC3_to_OMC4'] = beams['input']

    beams = OMC1.hitFromHR(b, order=1)
    b = beams['r1']   
    beam_dict['OMC4_to_OMC1'] = beams['input'] 
    beam_dict['OMC1_refl_s1'] = beams['s1']
    beam_dict['OMC1_refl_t1'] = beams['t1']

    # OMC input path
    b =  beam_dict['OMC1_to_OMC2'].copy()
    b.propagate(0.1)
    b.flip()
    beams = OMC1.hitFromHR(b)
    beam_dict['OMC1_input_reverse'] = beams['t1'].copy()
    b = beams['t1'].copy()
    b.propagate(0.1)
    b.flip()
    beams = OMC1.hitFromAR(b)

    beam_dict['OMC1_input'] = beams['input']
    beam_dict['OMC1_input_s1'] = beams['s1']
    
    return beam_dict

## Define mirrors

In [7]:
OMC1 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=12.7*mm, thickness=6.290147*mm,
                 wedgeAngle=deg2rad(1), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=OMC1_HR_Refl, Trans_HR=1-OMC1_HR_Refl,
                 Refl_AR=0.1*percent, Trans_AR=1-0.1*percent,
                 n=nsilica, name='OMC1', HRtransmissive=True)

OMC2 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=12.7*mm, thickness=6.290147*mm,
                 wedgeAngle=-deg2rad(1), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=OMC1_HR_Refl, Trans_HR=1-OMC1_HR_Refl,
                 Refl_AR=0.1*percent, Trans_AR=1-0.1*percent,
                 n=nsilica, name='OMC2', HRtransmissive=True)

OMC3 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=12.7*mm, thickness=6.4*mm,
                 wedgeAngle=0., inv_ROC_HR=1/Roc,
                 inv_ROC_AR=0.,
                 Refl_HR=OMC3_HR_Refl, Trans_HR=1-OMC3_HR_Refl,
                 Refl_AR=0.1*percent, Trans_AR=1-0.1*percent,
                 n=nsilica, name='OMC3', HRtransmissive=True)

OMC4 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=12.7*mm, thickness=6.4*mm,
                 wedgeAngle=0., inv_ROC_HR=1/Roc,
                 inv_ROC_AR=0.,
                 Refl_HR=OMC3_HR_Refl, Trans_HR=1-OMC3_HR_Refl,
                 Refl_AR=0.1*percent, Trans_AR=1-0.1*percent,
                 n=nsilica, name='OMC4', HRtransmissive=True)


## Place the mirrors

In [8]:
OMC_origin = np.array([32*mm, -67*mm])

OMC1_rel_pos = np.array([0., 0.])
OMC2_rel_pos = np.array([0., -0.357])
OMC3_rel_pos = np.array([0.0378945906, 0.011])
OMC4_rel_pos = np.array([0.0378945906, -0.368])


In [9]:
OMC1.HRcenter = OMC1_rel_pos + OMC_origin
OMC2.HRcenter = OMC2_rel_pos + OMC_origin
OMC3.HRcenter = OMC3_rel_pos + OMC_origin
OMC4.HRcenter = OMC4_rel_pos + OMC_origin

In [10]:
align_OMC()

## OMC eigen mode

In [11]:
(qOMCx, qOMCy, b) = OMC_eigen_mode()

## OMC beams

In [12]:
beam_dict = OMC_beams()
del beam_dict['OMC1_input']

# Outside of the OMC

## Define steering mirrors

In [13]:
OBS1_HR_Refl = 99.5*percent
OBS2_HR_Refl = 50*percent
OBS3_HR_Refl = 99.*percent
OBS4_HR_Refl = 50*percent
OBS5_HR_Refl = 99.95*percent
QPSTM1_HR_Refl = 99.95*percent
QPSTM2_HR_Refl = 99.95*percent

OBS3_ROC = 600*mm


OBS1 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=40*mm, thickness=10*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=OBS1_HR_Refl, Trans_HR=1-OBS1_HR_Refl,
                 Refl_AR=0.2*percent, Trans_AR=1-0.2*percent,
                 n=nsilica, name='OBS1', HRtransmissive=True)


OBS2 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=20*mm, thickness=8*mm,
                 wedgeAngle=deg2rad(-1), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=OBS2_HR_Refl, Trans_HR=1-OBS2_HR_Refl,
                 Refl_AR=0.2*percent, Trans_AR=1-0.2*percent,
                 n=nsilica, name='OBS2', HRtransmissive=True)

OBS3 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                    diameter=12.7*mm, thickness=6.35*mm,
                    wedgeAngle=deg2rad(0), inv_ROC_HR=1/OBS3_ROC,
                    inv_ROC_AR=0.,
                    Refl_HR=OBS3_HR_Refl, Trans_HR=1-OBS3_HR_Refl,
                    Refl_AR=1.*percent, Trans_AR=1-1.*percent,
                    n=nsilica, name='OBS3', HRtransmissive=True)

OBS4 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=20*mm, thickness=8*mm,
                 wedgeAngle=deg2rad(-1), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=OBS4_HR_Refl, Trans_HR=1-OBS4_HR_Refl,
                 Refl_AR=0.2*percent, Trans_AR=1-0.2*percent,
                 n=nsilica, name='OBS4', HRtransmissive=True)

OBS5 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=20*mm, thickness=8*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=OBS5_HR_Refl, Trans_HR=1-OBS5_HR_Refl,
                 Refl_AR=0.2*percent, Trans_AR=1-0.2*percent,
                 n=nsilica, name='OBS5', HRtransmissive=True)

QPSTM1 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=20*mm, thickness=8*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=QPSTM1_HR_Refl, Trans_HR=1-QPSTM1_HR_Refl,
                 Refl_AR=0.2*percent, Trans_AR=1-0.2*percent,
                 n=nsilica, name='QPSTM1', HRtransmissive=True)

QPSTM2 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=20*mm, thickness=8*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=QPSTM2_HR_Refl, Trans_HR=1-QPSTM2_HR_Refl,
                 Refl_AR=0.2*percent, Trans_AR=1-0.2*percent,
                 n=nsilica, name='QPSTM2', HRtransmissive=True)


## Define beam dumps

In [14]:
BD1 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=25*mm, thickness=3*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=1*percent,
                 Refl_AR=1*percent, Trans_AR=1*percent,
                 n=nsilica, name='BD1', HRtransmissive=True)

BD2 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=25*mm, thickness=3*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=1*percent,
                 Refl_AR=1*percent, Trans_AR=1*percent,
                 n=nsilica, name='BD2', HRtransmissive=True)

BD3 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=25*mm, thickness=3*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=1*percent,
                 Refl_AR=1*percent, Trans_AR=1*percent,
                 n=nsilica, name='BD3', HRtransmissive=True)

BD4 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=25*mm, thickness=3*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=1*percent,
                 Refl_AR=1*percent, Trans_AR=1*percent,
                 n=nsilica, name='BD4', HRtransmissive=True)

BD5 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=25*mm, thickness=3*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=1*percent,
                 Refl_AR=1*percent, Trans_AR=1*percent,
                 n=nsilica, name='BD5', HRtransmissive=True)

BD6 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=25*mm, thickness=3*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=1*percent,
                 Refl_AR=1*percent, Trans_AR=1*percent,
                 n=nsilica, name='BD6', HRtransmissive=True)

BD7 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=25*mm, thickness=3*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=1*percent,
                 Refl_AR=1*percent, Trans_AR=1*percent,
                 n=nsilica, name='BD7', HRtransmissive=True)

BD8 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=25*mm, thickness=3*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=1*percent,
                 Refl_AR=1*percent, Trans_AR=1*percent,
                 n=nsilica, name='BD8', HRtransmissive=True)

BD9 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=25*mm, thickness=3*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=1*percent,
                 Refl_AR=1*percent, Trans_AR=1*percent,
                 n=nsilica, name='BD9', HRtransmissive=True)

BD10 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=25*mm, thickness=3*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=1*percent,
                 Refl_AR=1*percent, Trans_AR=1*percent,
                 n=nsilica, name='BD10', HRtransmissive=True)

## Define PDs

In [15]:
DCPD1 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=3*mm, thickness=1*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=0,
                 Refl_AR=0.2*percent, Trans_AR=1-0.2*percent,
                 n=nsilica, name='DCPD1', HRtransmissive=False)

DCPD2 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=3*mm, thickness=1*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=0,
                 Refl_AR=0.2*percent, Trans_AR=1-0.2*percent,
                 n=nsilica, name='DCPD2', HRtransmissive=False)

QPD1 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=10*mm, thickness=3*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=0,
                 Refl_AR=0.2*percent, Trans_AR=1-0.2*percent,
                 n=nsilica, name='QPD1', HRtransmissive=False)

QPD2 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=10*mm, thickness=3*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=5*percent, Trans_HR=0,
                 Refl_AR=0.2*percent, Trans_AR=1-0.2*percent,
                 n=nsilica, name='QPD2', HRtransmissive=False)

## Put OBS1

In [16]:
OBS1.HRcenter = [32.*mm, -30.5*mm]
OBS1.normAngleHR = deg2rad(-134.6)

In [17]:
# Check the incident beam angle and position to the OMC breadboard from outside 
# These numbers should be roughly the same as before.

b = beam_dict['OMC1_input_reverse']
beams = OBS1.hitFromHR(b)
b = beams['r1']
print((rad2deg(b.dirAngle)-180-1.277499)/2)
print(b.pos*1000 - np.array([32.453522, -30.772137]))


-0.013125158758296451
[-0.31698385  0.13749209]


## Prepare input beam to the OMC BB

In [18]:
b = beam_dict['OMC1_input_reverse']
beams = OBS1.hitFromHR(b)
b = beams['r1']
b.propagate(1)
b.flip()
b.P = 1.0

beams = OBS1.hitFromHR(b)
beam_dict['Input_to_OMC_BB'] = beams['input']

b = beams['r1']
beams = OMC1.hitFromAR(b)
beam_dict['OBS1_to_OMC1'] = beams['input']

del beam_dict['OMC1_input_reverse']


## OMC REFL path

In [19]:
b = beam_dict['OMC1_refl_t1']
beams = OBS1.hitFromHR(b)
beam_dict['OMC1_to_OBS1_REFL'] = beams['input']
b = beams['r1']
b.length = 1.0
beam_dict['OBS1_to_REFL'] = b

del beam_dict['OMC1_refl_t1']

## Beam path for QPD

### Put OBS2, QPSTM1, QPSTM2

In [20]:
OBS2.HRcenter = [127*mm, -28*mm]
OBS2.normAngleHR = deg2rad(-133)
QPSTM1.HRcenter = [134*mm, -227*mm]
QPSTM1.normAngleHR = deg2rad(82.401502)
QPSTM2.HRcenter = [174*mm, -82*mm]
QPSTM2.normAngleHR = deg2rad(-97.598498)

### Adjust the angle of OBS2 iteratively

In [21]:
b = beam_dict['Input_to_OMC_BB'].copy()

beams = OBS1.hitFromHR(b)
beam_dict['OBS1_s1'] = beams['s1']
beam_to_OBS2 = beams['t1']


for ii in range(6):
    beams = OBS2.hitFromHR(beam_to_OBS2)
    b = beams['r1']

    v1 = normalize(QPSTM1.HRcenter - b.pos)

    dtheta = b.dirAngle - (np.arctan(v1[1]/v1[0])+2*pi)

    OBS2.normAngleHR = OBS2.normAngleHR - dtheta/2

print(dtheta)

beams = OBS2.hitFromHR(beam_to_OBS2)
beam_dict['OBS1_to_OBS2'] = beams['input']


1.234568003383174e-13


### QPD1

In [22]:
QPD1.HRcenter = [163.*mm, -23.*mm]
QPD1.normAngleHR = deg2rad(173)

beams = OBS2.hitFromHR(beam_to_OBS2)
beam_dict['OBS2_s1'] = beams['s1']
b = beams['t1']

beams = QPD1.hitFromHR(b)
QPD1.HRcenter = beams['r1'].pos

beams = QPD1.hitFromHR(b)
beam_dict['OBS2_to_QPD1'] = beams['input']
beams['r1'].length = 0.3
beam_dict['OBS2_r1'] = beams['r1']


### QPD2

In [23]:
QPD2.HRcenter = [177.*mm, -224.*mm]
QPD2.normAngleHR = deg2rad(80.401494)

beams = OBS2.hitFromHR(beam_to_OBS2)
b = beams['r1']

beams = QPSTM1.hitFromHR(b)
beam_dict['OBS2_to_QPSTM1'] = beams['input']
b = beams['r1']

v1 = - beam_dict['OBS2_to_QPSTM1'].dirVect
v2 = normalize(QPSTM2.HRcenter - b.pos)
QPSTM1.normVectHR = (v1 + v2)/2

beams = QPSTM1.hitFromHR(beam_dict['OBS2_to_QPSTM1'])
beam_dict['OBS2_to_QPSTM1'] = beams['input']
b = beams['r1']

v1 = normalize(QPSTM1.HRcenter - QPSTM2.HRcenter)
v2 = normalize(QPD2.HRcenter - QPSTM2.HRcenter)
QPSTM2.normVectHR = (v1 + v2)/2

beams = QPSTM2.hitFromHR(b)
beam_dict['QPSTM1_to_QPSTM2'] = beams['input']
b = beams['r1']

beams = QPD2.hitFromHR(b)
beam_dict['QPSTM2_to_QPD2'] = beams['input']
beam_dict['QPD2_r1'] = beams['r1']


## Output beam path

### OBS3 and BD9/10

In [24]:
OBS3.HRcenter = [32.*mm, -476*mm]
OBS3.normAngleHR = deg2rad(44.4)

OBS4.HRcenter = [127.*mm, -475.*mm]
OBS4.normAngleHR = deg2rad(135.4)

BD9.HRcenter = [113.*mm, -494.*mm]
BD9.normAngleHR = deg2rad(-153)

BD10.normVectHR = - BD9.normVectHR
v = vector_rotation_2D(BD9.normVectHR, deg2rad(14))
BD10.HRcenter = BD9.HRcenter + 6.5*mm*v
BD10.rotate(deg2rad(28))

beams = OMC2.hitFromHR(beam_dict['OMC1_to_OMC2'], order=3)

beam_dict['OMC2_s1'] = beams['s1']
beam_dict['OMC2_s2'] = beams['s2']
beam_dict['OMC2_s3'] = beams['s3']

b1 = beams['t1']
b2 = beams['t2']

beams = OBS3.hitFromHR(b1)
br1 = beams['r1']

beams = OBS3.hitFromHR(b2)
br2 = beams['r1']

OBS3.HRcenter = (br1.pos + br2.pos)/2

beams = OBS3.hitFromHR(b2)
beam_dict['OMC2_t2_to_OBS3'] = beams['input']
br2 = beams['r1']

beams = BD9.hitFromHR(br2)
beam_dict['OBS3_to_BD9'] = beams['input']
b = beams['r1']

beams = BD10.hitFromHR(b)
beam_dict['BD9_to_BD10'] = beams['input']
b = beams['r1']

beams = BD9.hitFromHR(b)
beam_dict['BD10_to_BD9'] = beams['input']

beams = OBS3.hitFromHR(b1)
beam_dict['OMC2_to_OBS3'] = beams['input']
beam_dict['OBS3_s1'] = beams['s1']
beams['t1'].length = 0.2
beam_dict['OBS3_t1'] = beams['t1']

br1 = beams['r1']


### OBS4 and BD5/6

In [25]:
DCPD1.HRcenter = [126.5*mm, -273*mm]
DCPD1.normAngleHR = deg2rad(-109)

# Iteratively adjust the angle of OBS4
for ii in range(6):
    beams = OBS4.hitFromHR(br1)
    b = beams['r1']
    v1 = normalize(br1.pos - b.pos)
    v2 = normalize(DCPD1.HRcenter - b.pos)
    OBS4.normVectHR = (v1 + v2)/2

beams = OBS4.hitFromHR(br1, order=3)
beam_dict['OBS3_to_OBS4'] = beams['input']
beam_dict['OBS4_s1'] = beams['s1']
beam_dict['OBS4_s2'] = beams['s2']
beam_dict['OBS4_s3'] = beams['s3']
br1 = beams['r1']
br2 = beams['r2']
beam_dict['OBS4_t1'] = beams['t1']
beam_dict['OBS4_t2'] = beams['t2']


BD6.HRcenter = [149.*mm - 0.5*mm, -380.*mm]
BD6.normAngleHR = deg2rad(160)

BD5.normVectHR = - BD6.normVectHR
v = vector_rotation_2D(BD6.normVectHR, deg2rad(-14))
BD5.HRcenter = BD6.HRcenter + 6.5*mm*v
BD5.rotate(deg2rad(-28))

beams = BD5.hitFromHR(br2)
beam_dict['OBS4_to_BD5'] = beams['input']
b = beams['r1']

beams = BD6.hitFromHR(b)
beam_dict['BD5_to_BD6'] = beams['input']


### DCPD1 and BD1/2

In [26]:
n = 3.5
rad2deg(np.arctan(n))

74.05460409907715

In [27]:
BD1.HRcenter = [96.*mm, -318.*mm]
BD1.normAngleHR = deg2rad(-1)

BD2.normVectHR = - BD1.normVectHR
v = vector_rotation_2D(BD1.normVectHR, deg2rad(-14))
BD2.HRcenter = BD1.HRcenter + 6.5*mm*v
BD2.rotate(deg2rad(-28))


beams = DCPD1.hitFromHR(br1)
beam_dict['OBS4_to_DCPD1'] = beams['input']
b = beams['r1']

beams = BD1.hitFromHR(b)
beam_dict['DCPD1_to_BD1'] = beams['input']
b = beams['r1']

beams = BD2.hitFromHR(b)
beam_dict['BD1_to_BD2'] = beams['input']


### OBS5 and BD7/8

In [28]:
OBS5.HRcenter = [164.5*mm, -478.5*mm]
OBS5.normAngleHR = deg2rad(136.)

DCPD2.HRcenter = [163.*mm, -305.*mm]
DCPD2.normAngleHR = deg2rad(-108.)

BD7.HRcenter = [179.5*mm, -408.5*mm]
BD7.normAngleHR = deg2rad(-48)

BD8.normVectHR = - BD7.normVectHR
v = vector_rotation_2D(BD7.normVectHR, deg2rad(14))
BD8.HRcenter = BD7.HRcenter + 6.5*mm*v
BD8.rotate(deg2rad(28))


v1 = - beam_dict['OBS4_t1'].dirVect
for ii in range(6):
    beams = OBS5.hitFromHR(beam_dict['OBS4_t1'])
    v2 = normalize(DCPD2.HRcenter - beams['r1'].pos)
    OBS5.normVectHR = (v1 + v2)/2

beams = OBS5.hitFromHR(beam_dict['OBS4_t1'])
beam_dict['OBS4_to_OBS5'] = beams['input']
beam_dict['OBS5_s1'] = beams['s1']
beams['t1'].length = 0.2
beam_dict['OBS5_t1'] = beams['t1']
br1 = beams['r1']

# BD7 path
beams = OBS5.hitFromHR(beam_dict['OBS4_t2'])
beam_dict['OBS4_t2_to_OBS5'] = beams['input']
br2 = beams['r1']

beams = BD7.hitFromHR(br2)
beam_dict['OBS5_to_BD7'] = beams['input']
b = beams['r1']

beams = BD8.hitFromHR(b)
beam_dict['BD7_to_BD8'] = beams['input']

### DCPD2 and BD3/4

In [29]:
BD3.HRcenter = [99.5*mm, -400.*mm]
BD3.normAngleHR = deg2rad(-1)

BD4.normVectHR = - BD3.normVectHR
v = vector_rotation_2D(BD3.normVectHR, deg2rad(-14))
BD4.HRcenter = BD3.HRcenter + 6.5*mm*v
BD4.rotate(deg2rad(-28))

beams = DCPD2.hitFromHR(br1)
beam_dict['OBS5_to_DCPD2'] = beams['input']
b = beams['r1']

beams = BD3.hitFromHR(b)
beam_dict['DCPD2_to_BD3'] = beams['input']
b = beams['r1']

beams = BD4.hitFromHR(b)
beam_dict['BD3_to_BD4'] = beams['input']

In [30]:
# beam_dict

# Save mirrors and beams

In [32]:
opticsList = [OMC1, OMC2, OMC3, OMC4, OBS1, OBS2, OBS3, OBS4, OBS5, QPSTM1, QPSTM2, DCPD1,
             DCPD2, QPD1, QPD2, BD1, BD2, BD3, BD4, BD5, BD6, BD7, BD8, BD9, BD10]
             
with open('KAGRA_OMC.pkl', 'wb') as f:
    _pickle.dump({'opticsList': opticsList, 
                  'beamDict': beam_dict},f)

# Draw optical layout

In [32]:
beamList = list(beam_dict.values())
# auxBeamList = list(auxBeamDict.values())

# tools.transAll(beamList, -OMC1.HRcenter)
# tools.transAll(auxBeamList, -OMC1.HRcenter)
# tools.transAll([OMC1, OMC2, OMC3, OMC4, OBS1, OBS3], -OMC1.HRcenter)

cnv = draw.Canvas()
cnv.add_layer("main_beam_x", color=(0,0,0))
cnv.add_layer("main_beam_x_width", color=(0,0,1))
cnv.add_layer("main_beam_y", color=(0,0,0))
cnv.add_layer("main_beam_y_width", color=(1,0,0))

#Draw the mirror
OMC1.draw(cnv, drawName=True)
OMC2.draw(cnv, drawName=True)
OMC3.draw(cnv, drawName=True)
OMC4.draw(cnv, drawName=True)
OBS1.draw(cnv, drawName=True)
OBS2.draw(cnv, drawName=True)
OBS3.draw(cnv, drawName=True)
OBS4.draw(cnv, drawName=True)
OBS5.draw(cnv, drawName=True)
QPSTM1.draw(cnv, drawName=True)
QPSTM2.draw(cnv, drawName=True)
DCPD1.draw(cnv, drawName=True)
DCPD2.draw(cnv, drawName=True)
QPD1.draw(cnv, drawName=True)
QPD2.draw(cnv, drawName=True)

BD1.draw(cnv, drawName=True)
BD2.draw(cnv, drawName=True)
BD3.draw(cnv, drawName=True)
BD4.draw(cnv, drawName=True)
BD5.draw(cnv, drawName=True)
BD6.draw(cnv, drawName=True)
BD7.draw(cnv, drawName=True)
BD8.draw(cnv, drawName=True)
BD9.draw(cnv, drawName=True)
BD10.draw(cnv, drawName=True)

#Draw beams
gtrace.draw.tools.drawAllBeams(cnv, beamList, drawWidth=True, sigma=3.0,  mode='x', layer='main_beam_x')

# Draw Bread Board
cnv.add_shape(draw.Rectangle(point=[0.0, -500*mm], width=200*mm, height=500*mm), layername='Bread Board')


#Save the result as a DXF file
renderer.renderDXF(cnv, 'OMC_Layout_O4.dxf')

# Non-sequential trace

In [33]:


input_beam = beam_dict['Input_to_OMC_BB'].copy()
input_beam.P = 1
# OMC1.HRtransmissive = False
OMC1.Trans_HR = 0.0
beams1 = non_seq_trace(opticsList, src_beam=input_beam, power_threshold=1.0/10000)

input_beam = beam_dict['OMC1_to_OMC2'].copy()
input_beam.P = 250
OMC1.Refl_HR = 0.0
OMC1.HRtransmissive = False

beams2 = non_seq_trace(opticsList, src_beam=input_beam, power_threshold=1.0/10000)



In [34]:
beamList = beams1 + beams2

cnv = draw.Canvas()
cnv.add_layer("main_beam_x", color=(0,0,0))
cnv.add_layer("main_beam_x_width", color=(0,0,1))
cnv.add_layer("main_beam_y", color=(0,0,0))
cnv.add_layer("main_beam_y_width", color=(1,0,0))

#Draw the mirror
OMC1.draw(cnv, drawName=True)
OMC2.draw(cnv, drawName=True)
OMC3.draw(cnv, drawName=True)
OMC4.draw(cnv, drawName=True)
OBS1.draw(cnv, drawName=True)
OBS2.draw(cnv, drawName=True)
OBS3.draw(cnv, drawName=True)
OBS4.draw(cnv, drawName=True)
OBS5.draw(cnv, drawName=True)
QPSTM1.draw(cnv, drawName=True)
QPSTM2.draw(cnv, drawName=True)
DCPD1.draw(cnv, drawName=True)
DCPD2.draw(cnv, drawName=True)
QPD1.draw(cnv, drawName=True)
QPD2.draw(cnv, drawName=True)

BD1.draw(cnv, drawName=True)
BD2.draw(cnv, drawName=True)
BD3.draw(cnv, drawName=True)
BD4.draw(cnv, drawName=True)
BD5.draw(cnv, drawName=True)
BD6.draw(cnv, drawName=True)
BD7.draw(cnv, drawName=True)
BD8.draw(cnv, drawName=True)
BD9.draw(cnv, drawName=True)
BD10.draw(cnv, drawName=True)

#Draw beams
gtrace.draw.tools.drawAllBeams(cnv, beamList, drawWidth=True, sigma=3.0,  mode='x', layer='main_beam_x', 
                               drawPower=True, fontSize=0.002)
gtrace.draw.tools.drawAllBeams(cnv, beamList, drawWidth=True, sigma=3.0,  mode='y', layer='main_beam_y')

# Draw Bread Board
cnv.add_shape(draw.Rectangle(point=[0.0, -500*mm], width=200*mm, height=500*mm), layername='Bread Board')

#Save the result as a DXF file
renderer.renderDXF(cnv, 'OMC_non_sq_trace_O4.dxf')

# List of mirrors and beams

## Optics List

In [35]:
with open('OpticsList.csv', mode='w') as f:
    f.write('OpticsName, Center coordinate, Normal Vector\n\n')
    for optic in opticsList:
        line = '{}, {}, {}\n'.format(optic.name, optic.HRcenter, optic.normVectHR)
        f.write(line)

## Main beam list

In [36]:
with open('MainBeamList.csv', mode='w') as f:
    f.write('Beam Name, Starting Point, Direction Vector, q-parameter in x, q-parameter in y\n\n')
    for name in list(beam_dict.keys()):
        b = beam_dict[name]
        line = '{}, {}, {}, {}, {}\n'.format(name, b.pos, b.dirVect, b.qx, b.qy)
        f.write(line)
    # for name in list(auxBeamDict.keys()):
    #     b = auxBeamDict[name]
    #     line = '{}, {}, {}, {}, {}\n'.format(name, b.pos, b.dirVect, b.qx, b.qy)
    #     f.write(line)        

## Stray beam list

In [783]:
with open('StrayBeamList.csv', mode='w') as f:
    f.write('Starting Point, Direction Vector, Power, q-parameter in x, q-parameter in y\n\n')
    for b in beamList:
        line = '{}, {}, {}, {}, {}\n'.format(b.pos, b.dirVect, b.P, b.qx, b.qy)
        f.write(line)

In [784]:
len(beamList)

57

# Checking Wedge Angle

In [60]:
#Put OMC1 at the origin

OMC1 = opt.Mirror(HRcenter=[0,0], normAngleHR=0.0,
                 diameter=12.7*mm, thickness=6.35*mm,
                 wedgeAngle=deg2rad(1), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=OMC1_HR_Refl, Trans_HR=1-OMC1_HR_Refl,
                 Refl_AR=0.1*percent, Trans_AR=1-0.1*percent,
                 n=nsilica, name='OMC1', HRtransmissive=True)

OMC1.HRcenter = [0,0]
#Face up
OMC1.normAngleHR = deg2rad(90)

#Laser source
q0 = gauss.Rw2q(np.inf, 1*mm)
b = beam.GaussianBeam(q0=q0, wl=1064*nm)
b.pos = [0, -1]
b.dirVect = [0, 1]

#Screen
SC = opt.Mirror(HRcenter=[0,-1], normVectHR=[0,1],
                 diameter=1000*mm, thickness=10*mm,
                 wedgeAngle=deg2rad(0), inv_ROC_HR=0.,
                 inv_ROC_AR=0.,
                 Refl_HR=1, Trans_HR=0,
                 Refl_AR=0.2*percent, Trans_AR=1-0.2*percent,
                 n=nsilica, name='Screen', HRtransmissive=True)


beams = OMC1.hitFromAR(b, order=1)
r1 = beams['r1']
r2 = beams['r2']

beams = SC.hitFromHR(r1)
bAR = beams['r1']

beams = SC.hitFromHR(r2)
bHR = beams['r1']

print('AR spot = {}mm, HR spot = {}mm'.format(bAR.pos[0]/mm, bHR.pos[0]/mm))
print('Beam separation = {}mm'.format((bAR.pos[0] - bHR.pos[0])/mm))


AR spot = 34.69902260547455mm, HR spot = -15.670426554871094mm
Beam separation = 50.369449160345646mm
